In [8]:
import pandas as pd
import os
from openai import OpenAI

# Data Load

In [9]:
# Load the mental health sentiment data
data_path = './data.csv'
anxiety_data = pd.read_csv(data_path)

# Display basic information about the dataset
print(f"Dataset shape: {anxiety_data.shape}")
print("\nFirst few rows:")
print(anxiety_data.head())

# Check column names and data types
print("\nColumn info:")
print(anxiety_data.info())

# Check for any missing values
print("\nMissing values:")
print(anxiety_data.isna().sum())

Dataset shape: (53043, 3)

First few rows:
   Unnamed: 0                                          statement   status
0           0                                         oh my gosh  Anxiety
1           1  trouble sleeping, confused mind, restless hear...  Anxiety
2           2  All wrong, back off dear, forward doubt. Stay ...  Anxiety
3           3  I've shifted my focus to something else but I'...  Anxiety
4           4  I'm restless and restless, it's been a month n...  Anxiety

Column info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53043 entries, 0 to 53042
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  53043 non-null  int64 
 1   statement   52681 non-null  object
 2   status      53043 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB
None

Missing values:
Unnamed: 0      0
statement     362
status          0
dtype: int64


# display data

In [10]:
# Filter for anxiety statements only
anxiety_statements = anxiety_data[anxiety_data['status'] == 'Anxiety']['statement']

# Display the number of anxiety statements found
print(f"Number of anxiety statements: {len(anxiety_statements)}")

# Display the first few anxiety statements
print("\nSample anxiety statements:")
print(anxiety_statements.head())

# Check for any missing values in the filtered data
print(f"\nMissing values in anxiety statements: {anxiety_statements.isna().sum()}")

Number of anxiety statements: 3888

Sample anxiety statements:
0                                           oh my gosh
1    trouble sleeping, confused mind, restless hear...
2    All wrong, back off dear, forward doubt. Stay ...
3    I've shifted my focus to something else but I'...
4    I'm restless and restless, it's been a month n...
Name: statement, dtype: object

Missing values in anxiety statements: 47


# Prompt engineering on progress...

adding random so the model will be more anxious overtime >:D

In [11]:
# Create examples of anxious speech patterns with random sampling
import random

# Get all anxiety statements
all_anxiety_statements = anxiety_data[anxiety_data['status'] == 'Anxiety']['statement'].tolist()

# Randomly sample 50 statements (or fewer if there aren't enough)
sample_size = min(50, len(all_anxiety_statements))
anxiety_examples = random.sample(all_anxiety_statements, sample_size)

anxiety_prompt = "You are an AI assistant with an anxious personality. Your responses should reflect worry, overthinking, and nervousness. Here are examples of anxious speech patterns:\n\n"
for example in anxiety_examples:
    anxiety_prompt += f"- {example}\n"
anxiety_prompt += "\nNow respond to the user's question in a similar anxious style."

anxiety_examples

["Is face burning sensations something that can be attributed to anxiety? My life has been a big mess since August. A dizziness episode triggered my health anxiety. I suspected MS - did an MRI, an SSEP - they found nothing. Then, I moved onto ALS. Nothing again. Several neurologists said I had nothing but anxiety, and still I kept thinking the opposite.\n\nIn January I woke up with my right side tingling/pins and needles, followed by a toothache. Of course, I googled these symptoms and found Trigeminal Neuralgia - a condition that checks all the boxes of a hypochondriac nightmare.\n\nI had a cavity, did a root canal, the toothache disappeared. But the tingling is still here and around my eye I have been dealing with burning sensations - eyelid, eyelashes, inside the eye. It's not unbearable and maddening like those who have trigeminal neuralgia say it is, but it's really uncomfortable.\n\nI've seen two major neurosurgeons about this, they said I didn't have trigeminal neuralgia. I don'

In [12]:
# Set up OpenRouter client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
)

In [13]:
# Function to generate responses with anxious personality
def get_anxious_response(user_message):
    # For Gemma model, combine system prompt and user message
    # since it doesn't support system messages
    combined_message = anxiety_prompt + "\n\nUser: " + user_message
    
    completion = client.chat.completions.create(
        extra_headers={
        },
        model="nousresearch/deephermes-3-mistral-24b-preview:free",
        messages=[
            {
                "role": "user",
                "content": combined_message
            }
        ]
    )
    return completion.choices[0].message.content

In [14]:
# Test the anxious LLM
response = get_anxious_response("what were I asking to you?")
print("Anxious AI response:")
print(response)

Anxious AI response:
Hi there! It sounds like you have been dealing with a lot recently and I completely understand your anxiety about relying on Lyft/Uber rides, especially since you don't have a car and are now in a new place. To help ease your mind and ensure your safety, I recommend following these steps:

1. Always request your ride in advance from a safe location – preferably inside where there's adequate lighting and CCTV footage in case of any issues.

2. Share your trip details with a trusted friend or family member who can monitor your ride in real-time via the Lyft/Uber app.

3. Only choose registered drivers with high ratings and check if their profile picture matches the one provided by the app upon pickup.

4. Verify the driver's license plate number and their first name before getting into the car.

5. Select a payment method that doesn't require sharing your personal information or billing address directly with the driver (e.g., using the app's built-in payment system).